1. The pdf act as the corpus to the llm model
2. This pdf is then converted into text chunks
3. Then the openAI Embeddings will convert these text chunks into vectors
4. Using vector search database to store these vectors for efficient queries

## Importing Libraries

In [11]:
import openai
import langchain
from langchain.document_loaders import PyPDFDirectoryLoader # to load pdf from directory
from langchain.text_splitter import RecursiveCharacterTextSplitter # for text splitting to remain inside the token size
from langchain.embeddings.openai import OpenAIEmbeddings # converting chunks into vectors
from langchain_pinecone import PineconeVectorStore,Pinecone # for creating a vector store
from langchain_openai import OpenAI

## Loading Environment variables

In [13]:
import os
from dotenv import load_dotenv
load_dotenv()

True

## Reading the PDF file

In [3]:
def read_doc(directory):
    file_loader=PyPDFDirectoryLoader(directory)
    documents=file_loader.load()
    return documents

In [4]:
doc=read_doc(os.path.join('documents'))

## Converting Document to Chunks

In [5]:
def chunk_data(docs,chunk_size=800,chunk_overlap=50):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    docs=text_splitter.split_documents(docs)
    return docs

In [6]:
documents=chunk_data(docs=doc)

## Embedding the documents

In [7]:
embeddings=OpenAIEmbeddings(api_key=os.environ['OPENAI_KEY'])

e:\Python Related Files\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [8]:
vectors=embeddings.embed_query("Hari Bol")

In [9]:
len(vectors)

1536

# pinecone.init(
#     api_key="15e9cde3-6af8-44d5-be34-88ec2c6c097e",
#     environment="gcp-starter"
# )

In [15]:
pinecone_api_key = os.environ.get('PINECONE_API_KEY')
pc = Pinecone(api_key=pinecone_api_key)
index_name="bhagwat-gita-search"

TypeError: Pinecone.__init__() got an unexpected keyword argument 'api_key'

In [ ]:
vectorStore=Pinecone.from_documents(doc,embeddings,index_name=index_name)

## Retrieve results from VectorDB

In [ ]:
def retrieve_query(query,k=2):
    matching_reults=vectorStore.similarity_search(query,k=k)
    return matching_reults

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain import OpenAI

In [ ]:
llm=OpenAI(model="text-davinci-003",temperature=0.5)
chain=load_qa_chain(llm,chain_type="stuff") # to create a Q&A application

## Making Queries to VectorDB

In [ ]:
def retrieve_answers(query):
    doc_search=retrieve_query(query)
    print(doc_search)
    response = chain.run(input_documents=doc_search,question=query)
    return response

In [ ]:
query="How to stay calm in tough situations"
answer=retrieve_answers(query=query)
print(answer)